## Építsünk egy logisztikus regressziós modellt - 4. lecke

![Logisztikus vs. lineáris regresszió infografika](../../../../../../2-Regression/4-Logistic/images/linear-vs-logistic.png)

#### **[Előadás előtti kvíz](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/15/)**

#### Bevezetés

Ebben az utolsó leckében a regresszióról, amely az egyik alapvető *klasszikus* gépi tanulási technika, a logisztikus regresszióval fogunk foglalkozni. Ezt a technikát akkor használjuk, amikor mintákat szeretnénk felfedezni bináris kategóriák előrejelzéséhez. Ez a cukorka csokoládé vagy sem? Ez a betegség fertőző vagy sem? Ez az ügyfél választja-e ezt a terméket vagy sem?

Ebben a leckében megtanulod:

-   A logisztikus regresszió technikáit

✅ Mélyítsd el a logisztikus regresszióval kapcsolatos tudásodat ebben a [Learn modulban](https://learn.microsoft.com/training/modules/introduction-classification-models/?WT.mc_id=academic-77952-leestott)

## Előfeltétel

A tök adatokkal való munka során már elég jól megismerkedtünk ahhoz, hogy felismerjük, van egy bináris kategória, amellyel dolgozhatunk: `Color`.

Építsünk egy logisztikus regressziós modellt, hogy megjósoljuk, adott változók alapján *milyen színű lehet egy adott tök* (narancssárga 🎃 vagy fehér 👻).

> Miért beszélünk bináris osztályozásról egy regresszióval kapcsolatos leckében? Csak nyelvi egyszerűség kedvéért, mivel a logisztikus regresszió [valójában egy osztályozási módszer](https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression), bár lineáris alapú. A következő leckecsoportban további adat-osztályozási módszerekről tanulhatsz.

Ehhez a leckéhez a következő csomagokra lesz szükségünk:

-   `tidyverse`: A [tidyverse](https://www.tidyverse.org/) egy [R csomagok gyűjteménye](https://www.tidyverse.org/packages), amely gyorsabbá, könnyebbé és szórakoztatóbbá teszi az adatelemzést!

-   `tidymodels`: A [tidymodels](https://www.tidymodels.org/) keretrendszer egy [csomaggyűjtemény](https://www.tidymodels.org/packages/) modellezéshez és gépi tanuláshoz.

-   `janitor`: A [janitor csomag](https://github.com/sfirke/janitor) egyszerű eszközöket kínál a piszkos adatok vizsgálatához és tisztításához.

-   `ggbeeswarm`: A [ggbeeswarm csomag](https://github.com/eclarke/ggbeeswarm) lehetőséget nyújt beeswarm-stílusú diagramok készítésére a ggplot2 használatával.

A csomagokat az alábbi módon telepítheted:

`install.packages(c("tidyverse", "tidymodels", "janitor", "ggbeeswarm"))`

Alternatívaként az alábbi szkript ellenőrzi, hogy megvannak-e a szükséges csomagok a modul elvégzéséhez, és telepíti őket, ha hiányoznak.


In [ ]:
suppressWarnings(if (!require("pacman"))install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, janitor, ggbeeswarm)


## **Határozd meg a kérdést**

A mi céljainkhoz ezt bináris formában fogjuk kifejezni: 'Fehér' vagy 'Nem fehér'. Az adatainkban van egy 'csíkos' kategória is, de kevés példa van rá, ezért nem fogjuk használni. Egyébként is eltűnik, amikor eltávolítjuk az üres értékeket az adatállományból.

> 🎃 Érdekesség: néha a fehér tököket 'szellem' tököknek hívjuk. Nem túl könnyű őket faragni, ezért nem olyan népszerűek, mint a narancssárgák, de nagyon menők! Így a kérdésünket átfogalmazhatnánk úgy is, hogy: 'Szellem' vagy 'Nem szellem'. 👻

## **A logisztikus regresszióról**

A logisztikus regresszió néhány fontos szempontból különbözik a lineáris regressziótól, amelyről korábban tanultál.

#### **Bináris osztályozás**

A logisztikus regresszió nem kínálja ugyanazokat a funkciókat, mint a lineáris regresszió. Az előbbi egy `bináris kategóriára` ("narancssárga vagy nem narancssárga") ad előrejelzést, míg az utóbbi képes `folyamatos értékeket` megjósolni, például a tök származási helye és betakarítási ideje alapján *mennyivel fog emelkedni az ára*.

![Infografika: Dasani Madipalli](../../../../../../2-Regression/4-Logistic/images/pumpkin-classifier.png)

### Egyéb osztályozások

A logisztikus regressziónak más típusai is vannak, például multinomiális és ordinális:

- **Multinomiális**, amely több kategóriát foglal magában - "Narancssárga, Fehér és Csíkos".

- **Ordinális**, amely rendezett kategóriákat tartalmaz, hasznos, ha logikusan szeretnénk sorrendbe állítani az eredményeket, például a tököket, amelyek egy véges számú méret szerint vannak rendezve (mini, kicsi, közepes, nagy, extra nagy, extra extra nagy).

![Multinomiális vs ordinális regresszió](../../../../../../2-Regression/4-Logistic/images/multinomial-vs-ordinal.png)

#### **A változóknak NEM kell korrelálniuk**

Emlékszel, hogy a lineáris regresszió jobban működött, ha a változók erősebben korreláltak? A logisztikus regresszió ennek az ellenkezője - a változóknak nem kell egymással összhangban lenniük. Ez jól működik az olyan adatokkal, amelyeknél a korrelációk viszonylag gyengék.

#### **Sok tiszta adatra van szükséged**

A logisztikus regresszió pontosabb eredményeket ad, ha több adatot használsz; a mi kis adatállományunk nem optimális ehhez a feladathoz, ezért ezt tartsd szem előtt.

✅ Gondold át, milyen típusú adatok alkalmasak a logisztikus regresszióra

## Gyakorlat - tisztítsd meg az adatokat

Először tisztítsd meg az adatokat egy kicsit, távolítsd el az üres értékeket, és válassz ki csak néhány oszlopot:

1. Add hozzá a következő kódot:


In [ ]:
# Load the core tidyverse packages
library(tidyverse)

# Import the data and clean column names
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv") %>% 
  clean_names()

# Select desired columns
pumpkins_select <- pumpkins %>% 
  select(c(city_name, package, variety, origin, item_size, color)) 

# Drop rows containing missing values and encode color as factor (category)
pumpkins_select <- pumpkins_select %>% 
  drop_na() %>% 
  mutate(color = factor(color))

# View the first few rows
pumpkins_select %>% 
  slice_head(n = 5)


Mindig megtekintheted az új adatkeretedet a [*glimpse()*](https://pillar.r-lib.org/reference/glimpse.html) függvény használatával, az alábbi módon:


In [ ]:
pumpkins_select %>% 
  glimpse()


Ellenőrizzük, hogy valóban egy bináris osztályozási problémát fogunk megoldani:


In [ ]:
# Subset distinct observations in outcome column
pumpkins_select %>% 
  distinct(color)


### Vizualizáció - kategóriális diagram
Mostanra újra betöltötted a tök adatokat, és megtisztítottad őket, hogy megmaradjon egy olyan adatállomány, amely néhány változót, például a Színt tartalmazza. Vizualizáljuk az adatkeretet a notebookban a ggplot könyvtár segítségével.

A ggplot könyvtár remek lehetőségeket kínál az adatok vizualizálására. Például összehasonlíthatod az adatok eloszlását minden Fajta és Szín esetében egy kategóriális diagramon.

1. Hozz létre egy ilyen diagramot a geombar függvény használatával, a tök adatainkat felhasználva, és adj meg egy színkódolást minden tök kategóriához (narancs vagy fehér):


In [ ]:
# Specify colors for each value of the hue variable
palette <- c(ORANGE = "orange", WHITE = "wheat")

# Create the bar plot
ggplot(pumpkins_select, aes(y = variety, fill = color)) +
  geom_bar(position = "dodge") +
  scale_fill_manual(values = palette) +
  labs(y = "Variety", fill = "Color") +
  theme_minimal()

Adatait megfigyelve látható, hogyan kapcsolódik a Szín adata a Fajtához.

✅ Ezen kategóriális ábra alapján milyen érdekes vizsgálatokat tudsz elképzelni?


### Adatelőkészítés: jellemzők kódolása

A tökök adatállományunkban minden oszlop sztring értékeket tartalmaz. A kategóriális adatokkal való munka intuitív az emberek számára, de nem a gépek számára. A gépi tanulási algoritmusok számokkal működnek jól. Ezért a kódolás nagyon fontos lépés az adatelőkészítési fázisban, mivel lehetővé teszi, hogy a kategóriális adatokat numerikus adatokká alakítsuk anélkül, hogy információt veszítenénk. A jó kódolás segít egy jó modell felépítésében.

A jellemzők kódolásához két fő típusú kódoló létezik:

1. Ordinális kódoló: jól alkalmazható ordinális változók esetében, amelyek olyan kategóriális változók, ahol az adatok logikai sorrendet követnek, mint például az `item_size` oszlop az adatállományunkban. Ez egy olyan leképezést hoz létre, amelyben minden kategóriát egy szám képvisel, amely a kategória sorrendjét jelöli az oszlopban.

2. Kategóriális kódoló: jól alkalmazható nominális változók esetében, amelyek olyan kategóriális változók, ahol az adatok nem követnek logikai sorrendet, mint például az összes `item_size`-től eltérő jellemző az adatállományunkban. Ez egy one-hot kódolás, ami azt jelenti, hogy minden kategóriát egy bináris oszlop képvisel: a kódolt változó értéke 1, ha a tök az adott Variety-hez tartozik, és 0, ha nem.

A Tidymodels egy újabb remek csomagot kínál: [recipes](https://recipes.tidymodels.org/) - egy csomag az adatok előfeldolgozásához. Meghatározunk egy `recipe`-t, amely specifikálja, hogy az összes prediktor oszlopot egész számokká kell kódolni, `prep`-eljük, hogy megbecsülje a szükséges mennyiségeket és statisztikákat bármely művelethez, majd végül `bake`, hogy az új adatokra alkalmazza a számításokat.

> Általában a recipes-t általában modellezés előfeldolgozójaként használják, ahol meghatározza, hogy milyen lépéseket kell alkalmazni egy adathalmazra annak érdekében, hogy készen álljon a modellezésre. Ebben az esetben **erősen ajánlott**, hogy a `workflow()`-t használja ahelyett, hogy manuálisan becsülné meg a receptet a prep és bake segítségével. Ezt mindjárt látni fogjuk.
>
> Azonban most a recipes + prep + bake kombinációt használjuk annak meghatározására, hogy milyen lépéseket kell alkalmazni egy adathalmazra annak érdekében, hogy készen álljon az adatelemzésre, majd kinyerjük az előfeldolgozott adatokat az alkalmazott lépésekkel.


In [ ]:
# Preprocess and extract data to allow some data analysis
baked_pumpkins <- recipe(color ~ ., data = pumpkins_select) %>%
  # Define ordering for item_size column
  step_mutate(item_size = ordered(item_size, levels = c('sml', 'med', 'med-lge', 'lge', 'xlge', 'jbo', 'exjbo'))) %>%
  # Convert factors to numbers using the order defined above (Ordinal encoding)
  step_integer(item_size, zero_based = F) %>%
  # Encode all other predictors using one hot encoding
  step_dummy(all_nominal(), -all_outcomes(), one_hot = TRUE) %>%
  prep(data = pumpkin_select) %>%
  bake(new_data = NULL)

# Display the first few rows of preprocessed data
baked_pumpkins %>% 
  slice_head(n = 5)


✅ Milyen előnyei vannak az ordinal encoder használatának az Item Size oszlophoz?

### Kapcsolatok elemzése a változók között

Miután előfeldolgoztuk az adatainkat, elemezhetjük a jellemzők és a címke közötti kapcsolatokat, hogy képet kapjunk arról, mennyire jól tudja a modell előre jelezni a címkét a jellemzők alapján. Az ilyen típusú elemzés legjobb módja az adatok ábrázolása. 
Ismét a ggplot geom_boxplot_ függvényt fogjuk használni, hogy kategóriális ábrán szemléltessük az Item Size, Variety és Color közötti kapcsolatokat. Az adatok jobb ábrázolása érdekében az Item Size kódolt oszlopát és a Variety nem kódolt oszlopát fogjuk használni.


In [ ]:
# Define the color palette
palette <- c(ORANGE = "orange", WHITE = "wheat")

# We need the encoded Item Size column to use it as the x-axis values in the plot
pumpkins_select_plot<-pumpkins_select
pumpkins_select_plot$item_size <- baked_pumpkins$item_size

# Create the grouped box plot
ggplot(pumpkins_select_plot, aes(x = `item_size`, y = color, fill = color)) +
  geom_boxplot() +
  facet_grid(variety ~ ., scales = "free_x") +
  scale_fill_manual(values = palette) +
  labs(x = "Item Size", y = "") +
  theme_minimal() +
  theme(strip.text = element_text(size = 12)) +
  theme(axis.text.x = element_text(size = 10)) +
  theme(axis.title.x = element_text(size = 12)) +
  theme(axis.title.y = element_blank()) +
  theme(legend.position = "bottom") +
  guides(fill = guide_legend(title = "Color")) +
  theme(panel.spacing = unit(0.5, "lines"))+
  theme(strip.text.y = element_text(size = 4, hjust = 0)) 


#### Használj rajzás diagramot

Mivel a Color egy bináris kategória (Fehér vagy Nem), 'egy [speciális megközelítést](https://github.com/rstudio/cheatsheets/blob/main/data-visualization.pdf) igényel a vizualizációhoz'.

Próbálj ki egy `rajzás diagramot`, hogy bemutasd a szín eloszlását az item_size függvényében.

Ehhez a [ggbeeswarm csomagot](https://github.com/eclarke/ggbeeswarm) fogjuk használni, amely módszereket kínál a méhraj-stílusú diagramok létrehozására a ggplot2 segítségével. A méhraj diagramok olyan módot kínálnak a pontok ábrázolására, amelyek normál esetben átfedésben lennének, így egymás mellé kerülnek.


In [ ]:
# Create beeswarm plots of color and item_size
baked_pumpkins %>% 
  mutate(color = factor(color)) %>% 
  ggplot(mapping = aes(x = color, y = item_size, color = color)) +
  geom_quasirandom() +
  scale_color_brewer(palette = "Dark2", direction = -1) +
  theme(legend.position = "none")


Most, hogy van némi elképzelésünk a színek bináris kategóriái és a méretek nagyobb csoportja közötti kapcsolatról, vizsgáljuk meg a logisztikus regressziót, hogy meghatározzuk egy adott tök valószínű színét.

## Építsd fel a modelledet

Válaszd ki azokat a változókat, amelyeket az osztályozási modelledben használni szeretnél, és oszd fel az adatokat tanuló és tesztelő halmazokra. Az [rsample](https://rsample.tidymodels.org/), a Tidymodels egyik csomagja, hatékony adatfelosztási és újramintavételi infrastruktúrát biztosít:


In [ ]:
# Split data into 80% for training and 20% for testing
set.seed(2056)
pumpkins_split <- pumpkins_select %>% 
  initial_split(prop = 0.8)

# Extract the data in each split
pumpkins_train <- training(pumpkins_split)
pumpkins_test <- testing(pumpkins_split)

# Print out the first 5 rows of the training set
pumpkins_train %>% 
  slice_head(n = 5)


🙌 Most már készen állunk arra, hogy egy modellt tanítsunk azáltal, hogy az edzési jellemzőket illesztjük az edzési címkéhez (szín).

Először készítünk egy receptet, amely meghatározza az adatainkon végrehajtandó előfeldolgozási lépéseket, hogy azokat modellezésre alkalmassá tegyük, például: a kategóriaváltozók egész számokká történő kódolását. Hasonlóan a `baked_pumpkins`-hez, létrehozunk egy `pumpkins_recipe`-t, de nem használjuk a `prep` és `bake` funkciókat, mivel ezeket egy munkafolyamatba fogjuk csomagolni, amit néhány lépés múlva látni fogsz.

A Tidymodels-ben számos módja van egy logisztikus regressziós modell meghatározásának. Lásd `?logistic_reg()`. Egyelőre az alapértelmezett `stats::glm()` motorral fogunk meghatározni egy logisztikus regressziós modellt.


In [ ]:
# Create a recipe that specifies preprocessing steps for modelling
pumpkins_recipe <- recipe(color ~ ., data = pumpkins_train) %>% 
  step_mutate(item_size = ordered(item_size, levels = c('sml', 'med', 'med-lge', 'lge', 'xlge', 'jbo', 'exjbo'))) %>%
  step_integer(item_size, zero_based = F) %>%  
  step_dummy(all_nominal(), -all_outcomes(), one_hot = TRUE)

# Create a logistic model specification
log_reg <- logistic_reg() %>% 
  set_engine("glm") %>% 
  set_mode("classification")


Most, hogy van egy receptünk és egy modell specifikációnk, meg kell találnunk egy módot arra, hogy ezeket egy objektumba csomagoljuk, amely először előkészíti az adatokat (a háttérben prep+bake), illeszti a modellt az előkészített adatokra, és lehetővé teszi az esetleges utófeldolgozási tevékenységeket is.

A Tidymodels-ben ez a praktikus objektum [`workflow`](https://workflows.tidymodels.org/) néven ismert, és kényelmesen tartalmazza a modellezési komponenseket.


In [ ]:
# Bundle modelling components in a workflow
log_reg_wf <- workflow() %>% 
  add_recipe(pumpkins_recipe) %>% 
  add_model(log_reg)

# Print out the workflow
log_reg_wf


Miután egy munkafolyamatot *meghatároztunk*, egy modell `betanítható` a [`fit()`](https://tidymodels.github.io/parsnip/reference/fit.html) függvény segítségével. A munkafolyamat kiértékeli a receptet és előfeldolgozza az adatokat a betanítás előtt, így nem kell manuálisan elvégeznünk ezt a prep és bake használatával.


In [ ]:
# Train the model
wf_fit <- log_reg_wf %>% 
  fit(data = pumpkins_train)

# Print the trained workflow
wf_fit


A modell nyomtatási eredményei megmutatják a tanítás során megtanult együtthatókat.

Most, hogy betanítottuk a modellt a tanító adatokkal, előrejelzéseket készíthetünk a teszt adatokra a [parsnip::predict()](https://parsnip.tidymodels.org/reference/predict.model_fit.html) segítségével. Kezdjük azzal, hogy a modellt használjuk a tesztkészlet címkéinek előrejelzésére, valamint az egyes címkékhez tartozó valószínűségek meghatározására. Ha a valószínűség nagyobb, mint 0.5, az előrejelzett osztály `WHITE`, különben `ORANGE`.


In [ ]:
# Make predictions for color and corresponding probabilities
results <- pumpkins_test %>% select(color) %>% 
  bind_cols(wf_fit %>% 
              predict(new_data = pumpkins_test)) %>%
  bind_cols(wf_fit %>%
              predict(new_data = pumpkins_test, type = "prob"))

# Compare predictions
results %>% 
  slice_head(n = 10)


Nagyszerű! Ez további betekintést nyújt abba, hogyan működik a logisztikus regresszió.

### Jobb megértés egy zavaró mátrix segítségével

Az egyes előrejelzések összehasonlítása a hozzájuk tartozó "valós" értékekkel nem túl hatékony módja annak, hogy meghatározzuk, mennyire jól teljesít a modell. Szerencsére a Tidymodels néhány további trükköt is tartogat: [`yardstick`](https://yardstick.tidymodels.org/) - egy csomag, amelyet a modellek hatékonyságának mérésére használnak teljesítménymutatók segítségével.

Az osztályozási problémákhoz kapcsolódó egyik teljesítménymutató a [`zavaró mátrix`](https://wikipedia.org/wiki/Confusion_matrix). A zavaró mátrix leírja, hogy egy osztályozási modell milyen jól teljesít. A zavaró mátrix összesíti, hogy az egyes osztályokban hány példát osztályozott helyesen a modell. Esetünkben megmutatja, hogy hány narancssárga tököt osztályoztak narancssárgának, és hány fehér tököt osztályoztak fehérnek; a zavaró mátrix azt is megmutatja, hogy hányat osztályoztak **rossz** kategóriákba.

A yardstick [**`conf_mat()`**](https://tidymodels.github.io/yardstick/reference/conf_mat.html) függvénye kiszámítja az előrejelzett és megfigyelt osztályok kereszt-táblázatát.


In [ ]:
# Confusion matrix for prediction results
conf_mat(data = results, truth = color, estimate = .pred_class)


Értelmezzük a zavaró mátrixot. A modellünknek az a feladata, hogy a tököket két bináris kategória között osztályozza: `fehér` és `nem-fehér`.

-   Ha a modell egy tököt fehérnek jósol, és az a valóságban is a 'fehér' kategóriába tartozik, akkor ezt `igaz pozitívnak` nevezzük, amit a bal felső szám mutat.

-   Ha a modell egy tököt nem fehérnek jósol, és az a valóságban a 'fehér' kategóriába tartozik, akkor ezt `hamis negatívnak` nevezzük, amit a bal alsó szám mutat.

-   Ha a modell egy tököt fehérnek jósol, és az a valóságban a 'nem-fehér' kategóriába tartozik, akkor ezt `hamis pozitívnak` nevezzük, amit a jobb felső szám mutat.

-   Ha a modell egy tököt nem fehérnek jósol, és az a valóságban a 'nem-fehér' kategóriába tartozik, akkor ezt `igaz negatívnak` nevezzük, amit a jobb alsó szám mutat.

| Valóság |
|:-------:|


|               |        |       |
|---------------|--------|-------|
| **Jósolt**    | FEHÉR | NARANCSSÁRGA |
| FEHÉR         | TP     | FP          |
| NARANCSSÁRGA  | FN     | TN          |

Ahogy valószínűleg kitaláltad, előnyös, ha az igaz pozitív és igaz negatív értékek száma nagyobb, míg a hamis pozitív és hamis negatív értékek száma kisebb, mivel ez azt jelzi, hogy a modell jobban teljesít.

A zavaró mátrix hasznos, mivel más mutatókat is létrehoz, amelyek segítenek jobban értékelni egy osztályozási modell teljesítményét. Nézzük meg ezeket:

🎓 Pontosság: `TP/(TP + FP)` A pozitívnak jósolt értékek közül hány valóban pozitív. Más néven [pozitív prediktív érték](https://en.wikipedia.org/wiki/Positive_predictive_value "Positive predictive value").

🎓 Visszahívás: `TP/(TP + FN)` Azoknak a pozitív eredményeknek az aránya, amelyek valóban pozitív mintákból származnak. Más néven `érzékenység`.

🎓 Specificitás: `TN/(TN + FP)` Azoknak a negatív eredményeknek az aránya, amelyek valóban negatív mintákból származnak.

🎓 Pontosság (Accuracy): `TP + TN/(TP + TN + FP + FN)` Azon címkék százaléka, amelyeket egy mintánál pontosan jósoltak meg.

🎓 F-mérték: A pontosság és a visszahívás súlyozott átlaga, ahol a legjobb érték 1, a legrosszabb pedig 0.

Számoljuk ki ezeket a mutatókat!


In [ ]:
# Combine metric functions and calculate them all at once
eval_metrics <- metric_set(ppv, recall, spec, f_meas, accuracy)
eval_metrics(data = results, truth = color, estimate = .pred_class)


## Ennek a modellnek az ROC görbéjének megjelenítése

Készítsünk még egy vizualizációt, hogy megnézzük az úgynevezett [`ROC görbét`](https://en.wikipedia.org/wiki/Receiver_operating_characteristic):


In [ ]:
# Make a roc_curve
results %>% 
  roc_curve(color, .pred_ORANGE) %>% 
  autoplot()


A ROC görbék gyakran használatosak annak szemléltetésére, hogy egy osztályozó milyen teljesítményt nyújt a valódi és hamis pozitív találatok tekintetében. A ROC görbék általában a `True Positive Rate`/Érzékenységet ábrázolják az Y tengelyen, míg a `False Positive Rate`/1-Specificitást az X tengelyen. Ezért a görbe meredeksége és a középvonal és a görbe közötti tér fontos: olyan görbét szeretnénk, amely gyorsan felfelé és a vonal fölé halad. A mi esetünkben kezdetben vannak hamis pozitív találatok, majd a vonal megfelelően felfelé és fölé halad.

Végül használjuk a `yardstick::roc_auc()` függvényt a görbe alatti terület (Area Under the Curve) kiszámításához. Az AUC egyik értelmezési módja az, hogy ez annak a valószínűsége, hogy a modell egy véletlenszerűen kiválasztott pozitív példát magasabbra rangsorol, mint egy véletlenszerűen kiválasztott negatív példát.


In [ ]:
# Calculate area under curve
results %>% 
  roc_auc(color, .pred_ORANGE)


Az eredmény körülbelül `0.975`. Mivel az AUC 0 és 1 között mozog, magas értéket szeretnél elérni, hiszen egy modell, amely 100%-ban helyes előrejelzéseket ad, AUC értéke 1 lesz; ebben az esetben a modell *elég jó*.

A jövőbeli leckékben a klasszifikációkról megtanulod, hogyan javíthatod a modell pontszámait (például az egyensúlyhiányos adatok kezelésével ebben az esetben).

## 🚀Kihívás

A logisztikus regresszióval kapcsolatban még sok felfedeznivaló van! De a legjobb módja a tanulásnak az, ha kísérletezel. Keress egy adathalmazt, amely alkalmas erre a típusú elemzésre, és építs egy modellt vele. Mit tanulsz? Tipp: próbáld ki [Kaggle](https://www.kaggle.com/search?q=logistic+regression+datasets) oldalt érdekes adathalmazokért.

## Áttekintés és önálló tanulás

Olvasd el [ennek a Stanford-i tanulmánynak](https://web.stanford.edu/~jurafsky/slp3/5.pdf) az első néhány oldalát a logisztikus regresszió gyakorlati alkalmazásairól. Gondolkodj el azon, hogy mely feladatok alkalmasabbak az egyik vagy másik típusú regressziós feladatra, amelyeket eddig tanulmányoztunk. Mi működne a legjobban?



---

**Felelősségkizárás**:  
Ez a dokumentum az [Co-op Translator](https://github.com/Azure/co-op-translator) AI fordítási szolgáltatás segítségével készült. Bár törekszünk a pontosságra, kérjük, vegye figyelembe, hogy az automatikus fordítások hibákat vagy pontatlanságokat tartalmazhatnak. Az eredeti dokumentum az eredeti nyelvén tekintendő hiteles forrásnak. Kritikus információk esetén javasolt a professzionális, emberi fordítás igénybevétele. Nem vállalunk felelősséget a fordítás használatából eredő félreértésekért vagy téves értelmezésekért.
